In [1]:
using PowerSystems
using Dates
using TimeSeries
using DataFrames
using CSV
using DataStructures
using PowerSystemCaseBuilder
using PowerSimulations
using HiGHS
using HydroPowerSimulations

In [2]:
# from https://github.com/NREL-Sienna/PowerSystems.jl/blob/main/docs/src/tutorials/creating_system.md
sys = System(100.0)

bus1 = ACBus(;
    number = 1,
    name = "bus1",
    bustype = ACBusTypes.REF,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

bus2 = ACBus(;
    number = 2,
    name = "bus2",
    bustype = ACBusTypes.PV,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

bus3 = ACBus(;
    number = 3,
    name = "bus3",
    bustype = ACBusTypes.PV,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

bus4 = ACBus(;
    number = 4,
    name = "bus4",
    bustype = ACBusTypes.PV,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

line21 = Line(;
    name = "line21",
    available = true,
    active_power_flow = 0.0,
    reactive_power_flow = 0.0,
    arc = Arc(; from = bus2, to = bus1),
    r = 0.00281, # Per-unit
    x = 0.0281, # Per-unit
    b = (from = 0.00356, to = 0.00356), # Per-unit
    rating = 6,
    angle_limits = (min = -0.7, max = 0.7),
);

line31 = Line(;
    name = "line31",
    available = true,
    active_power_flow = 0.0,
    reactive_power_flow = 0.0,
    arc = Arc(; from = bus3, to = bus1),
    r = 0.00281, # Resistance, Per-unit
    x = 0.0281, # Reactance, Per-unit
    b = (from = 0.00356, to = 0.00356), # Shunt susceptance, Per-unit
    rating = 4.0, # Line rating of 200 MVA / System base of 100 MVA
    angle_limits = (min = -0.7, max = 0.7),
);

line41 = Line(;
    name = "line41",
    available = true,
    active_power_flow = 0.0,
    reactive_power_flow = 0.0,
    arc = Arc(; from = bus4, to = bus1),
    r = 0.00281, # Resistance, Per-unit
    x = 0.0281, # Reactance, Per-unit
    b = (from = 0.00356, to = 0.00356), # Shunt susceptance, Per-unit
    rating = 4.0, # Line rating of 200 MVA / System base of 100 MVA
    angle_limits = (min = -0.7, max = 0.7),
);

load =  PowerLoad(;
    name = "load",
    available = true,
    bus = bus1,
    active_power = 0.0, # Per-unitized by device base_power
    reactive_power = 0.0, # Per-unitized by device base_power
    base_power = 110.0, # MVA
    max_active_power = 1.0, # 10 MW per-unitized by device base_power
    max_reactive_power = 0.0,
);

# https://nrel-sienna.github.io/PowerSystems.jl/stable/model_library/generated_RenewableDispatch/#RenewableDispatch
solar = RenewableDispatch(;
    name = "solar",
    available = true,
    bus = bus2,
    active_power = 0.0, # Per-unitized by device base_power
    reactive_power = 0.0, # Per-unitized by device base_power
    rating = 1.0, # per-unitized by device base_power
    prime_mover_type = PrimeMovers.PVe,
    reactive_power_limits = (min = 0.0, max = 0.0), # per-unitized by device base_power
    power_factor = 1.0,
    operation_cost = RenewableGenerationCost(CostCurve(LinearCurve(0))),
    # RenewableGenerationCost(CostCurve(LinearCurve(0.0075))),
    # https://nrel-sienna.github.io/PowerSystems.jl/stable/model_library/renewable_generation_cost/#RenewableGenerationCost
    # https://nrel-sienna.github.io/PowerSystems.jl/stable/api/public/#InfrastructureSystems.CostCurve
    # https://nrel-sienna.github.io/PowerSystems.jl/stable/api/public/#InfrastructureSystems.LinearCurve
    base_power = 248.212, # MVA
);

# https://github.com/NREL-Sienna/PowerSystems.jl/blob/ec2c55991de76d25deb0c9ac23d85e44979d9694/src/models/generated/EnergyReservoirStorage.jl#L8
battery_solar = EnergyReservoirStorage(;
    name = "battery_solar",
    prime_mover_type = PrimeMovers.BA,
    storage_technology_type = StorageTech.LIB,
    available = true,
    bus = bus2,
    storage_capacity = 1312, # MWh
    storage_level_limits = (min = 5.0 / 100.0, max = 100.0 / 100.0),
    initial_storage_capacity_level = 50.0 / 100.0,
    rating = 1, #Value in per_unit of the system
    active_power = 0, # Initial active power set point of the unit in MW. For power flow, this is the steady state operating point of the system. For production cost modeling, this may or may not be used as the initial starting point for the solver, depending on the solver used
    input_active_power_limits = (min = 0.0, max = 50.0), # Minimum and maximum limits on the input active power (i.e., charging), validation range: `(0, nothing)`
    output_active_power_limits = (min = 0.0, max = 50.0), # Minimum and maximum limits on the output active power (i.e., discharging), validation range: `(0, nothing)`
    reactive_power = 0.0, # Initial reactive power set point of the unit (MVAR), validation range: `reactive_power_limits`
    reactive_power_limits = (min = -50.0, max = 50.0), # Minimum and maximum reactive power limits. Set to `Nothing` if not applicable
    efficiency = (in = 0.80, out = 0.90),
    base_power = 100.0, # Base power of the unit (MVA) for [per unitization]
    operation_cost = StorageCost(charge_variable_cost=CostCurve(LinearCurve(0)), discharge_variable_cost=CostCurve(LinearCurve(0)))) 

wind = RenewableDispatch(;
    name = "wind",
    available = false,
    bus = bus3,
    active_power = 0.0,
    reactive_power = 0.0,
    rating = 1.0,
    prime_mover_type = PrimeMovers.WT,
    reactive_power_limits = (min = 0.0, max = 0.0),
    power_factor = 1.0,
    operation_cost = RenewableGenerationCost(CostCurve(LinearCurve(0))),
    base_power = 61.0 # MVA
    )

# https://github.com/NREL-Sienna/PowerSystems.jl/blob/ec2c55991de76d25deb0c9ac23d85e44979d9694/src/models/generated/EnergyReservoirStorage.jl#L8
battery_wind = EnergyReservoirStorage(;
    name = "battery_wind",
    prime_mover_type = PrimeMovers.BA,
    storage_technology_type = StorageTech.LIB,
    available = true,
    bus = bus3,
    storage_capacity = 1312, # MWh
    storage_level_limits = (min = 5.0 / 100.0, max = 100.0 / 100.0),
    initial_storage_capacity_level = 50.0 / 100.0,
    rating = 1, #Value in per_unit of the system
    active_power = 0, # Initial active power set point of the unit in MW. For power flow, this is the steady state operating point of the system. For production cost modeling, this may or may not be used as the initial starting point for the solver, depending on the solver used
    input_active_power_limits = (min = 0.0, max = 50.0), # Minimum and maximum limits on the input active power (i.e., charging), validation range: `(0, nothing)`
    output_active_power_limits = (min = 0.0, max = 50.0), # Minimum and maximum limits on the output active power (i.e., discharging), validation range: `(0, nothing)`
    reactive_power = 0.0, # Initial reactive power set point of the unit (MVAR), validation range: `reactive_power_limits`
    reactive_power_limits = (min = -50.0, max = 50.0), # Minimum and maximum reactive power limits. Set to `Nothing` if not applicable
    efficiency = (in = 0.80, out = 0.90),
    base_power = 100.0,
    operation_cost = StorageCost(charge_variable_cost=CostCurve(LinearCurve(0)), discharge_variable_cost=CostCurve(LinearCurve(0)))) # Base power of the unit (MVA) for [per unitization]

# https://github.com/NREL-Sienna/PowerSystems.jl/blob/ec2c55991de76d25deb0c9ac23d85e44979d9694/src/models/generated/HydroEnergyReservoir.jl
hydro = HydroPumpedStorage(;
    name = "hydro",
    available = true,
    bus = bus4,
    active_power = 0.0,
    reactive_power = 0.0,
    active_power_limits = (min=0.0, max=42), # MW from 2 28,000 HP pumps https://www.sdcwa.org/wp-content/uploads/2020/11/lake_hodges_fs.pdf?q=/sites/default/files/files/publications/lakehodges-fs.pdf&
    rating = 1.0, # "Maximum power withdrawal (MVA) of the pump"
    prime_mover_type = PrimeMovers.PS,
    reactive_power_limits = (min=0.0, max=42),
    rating_pump = 42, # MVA
    active_power_limits_pump = (min=0.0, max=42),
    reactive_power_limits_pump = (min=0.0, max=42),
    ramp_limits_pump = (up=42, down=42), # "ramp up and ramp down limits in MW/min of pump"
    time_limits_pump = (up=0.0, down=0), # "Minimum up and Minimum down time limits of pump in hours"
    inflow = 0, # "Baseline inflow into the upper reservoir (units can be p.u. or m^3/hr)"
    outflow = 0, # "Baseline outflow from the lower reservoir (units can be p.u. or m^3/hr)"
    initial_storage = (up=37314003, down=0), # "Initial storage capacity in the upper and lower reservoir (units can be p.u-hr or m^3)"
    ramp_limits=(up=42, down=42), # "ramp up and ramp down limits in MW/min"
    time_limits=(up=0.0, down=0), # "Minimum up and Minimum down time limits in hours"
    operation_cost = HydroGenerationCost(CostCurve(LinearCurve(0)), 0),
    base_power = 60.0, # MVA
    storage_capacity = (up=37314003, down=37314003) # m^3, from https://www.sandiego.gov/reservoirs-lakes/hodges-reservoir
    )

add_components!(sys, [bus1, bus2, bus3, bus4, line21, line31, line41, load, solar, battery_solar, wind, hydro])

In [3]:
# sys

In [ ]:
# get_component(PowerSystems.ElectricLoad, sys, "bus1")
# get_component(PowerSystems.RenewableDispatch, sys, "solar")
# get_component(PowerSystems.HydroPumpedStorage, sys, "hydro")

https://nrel-sienna.github.io/PowerSystems.jl/stable/how_to/parse_ts_from_csvs/

In [4]:
fname = joinpath(homedir(), "ecen5407", "ecen5407_project2", "data", "Sienna_Inputs", "sienna_pointers.json");
add_time_series!(sys, fname)

3-element Vector{TimeSeriesKey}:
 StaticTimeSeriesKey(SingleTimeSeries, "max_active_power", DateTime("2024-11-14T00:00:00"), Millisecond(300000), 105120, Dict{String, Any}())
 StaticTimeSeriesKey(SingleTimeSeries, "max_active_power", DateTime("2024-11-14T00:00:00"), Millisecond(300000), 105120, Dict{String, Any}())
 StaticTimeSeriesKey(SingleTimeSeries, "max_active_power", DateTime("2024-11-14T00:00:00"), Millisecond(300000), 105120, Dict{String, Any}())

In [5]:
transform_single_time_series!(
    sys,
    Dates.Hour(24), # horizon
    Dates.Hour(1), # interval
);

In [6]:
solver = optimizer_with_attributes(HiGHS.Optimizer, "mip_rel_gap" => 0.5);
template_uc = template_unit_commitment();
problem = DecisionModel(template_uc, sys; optimizer = solver, name = "UC", horizon = Hour(1))
build!(problem, output_dir = mktempdir())

InfrastructureSystems.Optimization.ModelBuildStatusModule.ModelBuildStatus.BUILT = 0

In [7]:
solve!(problem)

┌ Error: Optimizer returned NO_SOLUTION after 2 optimize! attempts
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/qdSY6/src/core/optimization_container.jl:792
┌ Error: Serializing Infeasible Problem at /tmp/jl_ns5Rlx/infeasible_UC.json
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/qdSY6/src/operation/operation_model_interface.jl:115
┌ Error: Decision Problem solve failed
│   exception =
│    Solving model UC failed at 2024-11-14T00:00:00
│    Stacktrace:
│      [1] error(s::String)
│        @ Base ./error.jl:35
│      [2] solve_impl!(model::DecisionModel{GenericOpProblem})
│        @ PowerSimulations ~/.julia/packages/PowerSimulations/qdSY6/src/operation/operation_model_interface.jl:118
│      [3] macro expansion
│        @ ~/.julia/packages/PowerSimulations/qdSY6/src/operation/decision_model.jl:494 [inlined]
│      [4] macro expansion
│        @ ~/.julia/packages/TimerOutputs/NRdsv/src/TimerOutput.jl:237 [inlined]
│      [5] (::PowerSimulations.var"#83#84"{Bool, Boo

InfrastructureSystems.Simulation.RunStatusModule.RunStatus.FAILED = 2

In [ ]:
res = OptimizationProblemResults(problem)

In [ ]:
renewable_param = read_parameter(res, "ActivePowerTimeSeriesParameter__RenewableDispatch")